In [16]:
from bitstring import BitArray, BitStream
from h264bits import H264Bits

In [17]:
from nalsps import SPS
from nalpps import PPS
from nalslice import Slice

In [18]:
input_file = open("data/baseline.264", "rb")
nalus_ba = list(BitArray(input_file).split('0x000001', bytealigned=True))[1:]

In [19]:
ppss = []
slices = []

In [20]:
for nalu_ba in nalus_ba:
    print(nalu_ba)
    # enulation_prevention_three_byte
    nalu_ba.replace('0x000003', '0x0000', bytealigned=True)
    nalu_bs = BitStream(nalu_ba)
    nb = H264Bits(nalu_bs)
    params = {"forbidden_zero_bit" : nb.f(1),
              "nal_ref_idc" : nb.u(2),
              "nal_unit_type" : nb.u(5)}
    print(params)
    # if params["nal_unit_type"] == 7: # SPS
    #     sps = SPS(nb, params = params)
    #     # dump_params(sps, "sps.json")
    # elif params["nal_unit_type"] == 8: # PPS
    #     pps = PPS(nb, sps = sps, params = params)
    #     ppss.append(pps)
    #     fname = "pps_" + str(len(ppss)) + ".json"
    #     # dump_params(pps, fname)
    # elif params["nal_unit_type"] in [1, 5]: # Slice
    #     slice = Slice(nb, sps = sps, ppss = ppss, params = params)
    #     # decode_slice(slice)
    #     break
    #     slices.append(slice)
    #     dump_mbs(slice, "slice_" + str(len(slices)) + "_mb.json")
    #     fname = "slice_" + str(len(slices)) + ".json"
    #     dump_params(slice, fname)
    # else:
    #     print("Unknown Slice type, ignore...")


0x0000016742c00bd902c4e840000003004000000c83c50a9200
{'forbidden_zero_bit': 0, 'nal_ref_idc': 3, 'nal_unit_type': 7}
0x00000168cb83cb20
{'forbidden_zero_bit': 0, 'nal_ref_idc': 3, 'nal_unit_type': 8}
0x0000010605ffff6ddc45e9bde6d948b7962cd820d923eeef78323634202d20636f7265203134382072323639392061356530366239202d20482e3236342f4d5045472d342041564320636f646563202d20436f70796c65667420323030332d32303136202d20687474703a2f2f7777772e766964656f6c616e2e6f7267...
{'forbidden_zero_bit': 0, 'nal_ref_idc': 0, 'nal_unit_type': 6}
0x000001658884087c41c45e040fbbf0031c63dd53d4a1fde892ea3b0b62fe7c9c327ce9b6e15234897500b8c2fe108e931af979c08ccf08fa4d4e9abf54697960724181546d27cb9c0829d46272ff8fdddde729670b6d3dc0b53d0b2c852f6f7c155d81823de200e4cdc6462a47c6406b49dce483962b8d1fa6edf0330c...
{'forbidden_zero_bit': 0, 'nal_ref_idc': 3, 'nal_unit_type': 5}
0x000001419a3812e69fae0b87f260be6c6a19d57f96ba5e5fc8bfdf74d72a2df26259d55f97ac4dd562e5957d7aa0bc718b3625d608440b21a979ecf7d606038bb4d7a97d1e0b4a6c432eb088993f24

The `forbidden_zero_bit` is always set to zero and is used to check for bit errors or synchronization issues. If this bit is not zero, it indicates an error in the stream.

The `nal_ref_idc` field in the NALU (Network Abstraction Layer Unit) header in H.264 video coding is a 2-bit field. Therefore, it can take on the following four possible values:

- 0: The NALU does not contain a sequence or picture parameter set, slice of a reference picture, or a slice data partition of a reference picture.
- 1: The NALU contains a slice of a reference picture or a slice data partition of a reference picture, but not a sequence or picture parameter set.
- 2: The NALU contains a sequence or picture parameter set, but not a slice of a reference picture or a slice data partition of a reference picture.
- 3: The NALU contains a sequence or picture parameter set, a slice of a reference picture, or a slice data partition of a reference picture.

These values are used to indicate the importance of the NALU in terms of error recovery and the decoding process. Higher values indicate higher importance.

In H.264 video coding, the `nal_unit_type` field in the NALU (Network Abstraction Layer Unit) header specifies the type of RBSP (Raw Byte Sequence Payload) data structure contained in the NALU. Here are the possible NAL unit types:

- 0: Unspecified
- 1: Coded slice of a non-IDR picture
- 2: Coded slice data partition A
- 3: Coded slice data partition B
- 4: Coded slice data partition C
- 5: Coded slice of an IDR picture
- 6: Supplemental enhancement information (SEI)
- 7: Sequence parameter set
- 8: Picture parameter set
- 9: Access unit delimiter
- 10: End of sequence
- 11: End of stream
- 12: Filler data
- 13: Sequence parameter set extension
- 14: Prefix NAL unit
- 15: Subset sequence parameter set
- 16-18: Reserved
- 19: Coded slice of an auxiliary coded picture without partitioning
- 20: Coded slice extension
- 21: Coded slice extension for depth view components
- 22-23: Reserved
- 24: STAP-A Single-time aggregation packet
- 25: STAP-B Single-time aggregation packet
- 26: MTAP16 Multi-time aggregation packet
- 27: MTAP24 Multi-time aggregation packet
- 28: FU-A Fragmentation unit
- 29: FU-B Fragmentation unit
- 30-31: Unspecified

Each of these types corresponds to a different kind of data or operation in the H.264 video stream.

In [21]:
print(sps)

None


**RBSP (Raw Byte Sequence Payload)**: Contains syntax structures such as SPS, PPS, slice data, etc., with Emulation Prevention Bytes (EPBs) removed.

In [26]:
sps = {}

In [30]:
# SPS
nalu_ba = nalus_ba[0]
print(nalu_ba)
nalu_ba.replace('0x000003', '0x0000', bytealigned=True)
nalu_bs = BitStream(nalu_ba)
nb = H264Bits(nalu_bs)
params = {"forbidden_zero_bit" : nb.f(1),
            "nal_ref_idc" : nb.u(2),
            "nal_unit_type" : nb.u(5)}
print(params)

# profile_idc = 66, 77, 88, 100
sps['profile_idc'] = nb.u(8)
constraint_set0_flag = nb.u(1)
constraint_set1_flag = nb.u(1)
constraint_set2_flag = nb.u(1)
constraint_set3_flag = nb.u(1)
constraint_set4_flag = nb.u(1)
constraint_set5_flag = nb.u(1)
reserved_zero_2bits = nb.u(2)
sps['level_idc'] = nb.u(8)
sps['seq_parameter_set_id'] =  nb.ue()
log2_max_frame_num_minus4 = nb.ue()
pic_order_cnt_type = nb.ue()
if pic_order_cnt_type == 0 :
    log2_max_pic_order_cnt_lsb_minus4 = nb.ue()
elif pic_order_cnt_type == 1:
    delta_pic_order_always_zero_flag = nb.u(1)
    offset_for_non_ref_pic = nb.se()
    offset_for_top_to_bottom_field = nb.se()
    num_ref_frames_in_pic_order_cnt_cycle = nb.ue()
    offset_for_ref_frame = []
    for i in range(num_ref_frames_in_pic_order_cnt_cycle):
        offset_for_ref_frame.append(se())
max_num_ref_frames = nb.ue()
gaps_in_frame_num_value_allowed_flag = nb.u(1)
pic_width_in_mbs_minus1 = nb.ue()
pic_height_in_map_units_minus1 = nb.ue()
frame_mbs_only_flag = nb.u(1)
if not frame_mbs_only_flag:
    mb_adaptive_frame_field_flag = nb.u(1)
else:
    mb_adaptive_frame_field_flag = 0
direct_8x8_inference_flag = nb.u(1)
frame_cropping_flag = nb.u(1)
if frame_cropping_flag:
    frame_crop_left_offset = nb.ue()
    frame_crop_right_offset = nb.ue()
    frame_crop_top_offset = nb.ue()
    frame_crop_bottom_offset = nb.ue()
vui_parameters_present_flag = nb.u(1)
if vui_parameters_present_flag:
#   vui_parameters
    aspect_ratio_info_present_flag = nb.u(1)
    if aspect_ratio_info_present_flag:
        aspect_ratio_idc = nb.u(8)
        if aspect_ratio_idc == Extended_SAR:
            sar_width = nb.u(16)
            sar_height = nb.u(16)
    overscan_info_present_flag = nb.u(1)
    if overscan_info_present_flag:
        overscan_appropriate_flag = nb.u(1)
    video_signal_type_present_flag = nb.u(1)
    if video_signal_type_present_flag:
        video_format = nb.u(3)
        video_full_range_flag = nb.u(1)
        colour_description_present_flag = nb.u(1)
        if colour_description_present_flag:
            colour_primaries = nb.u(8)
            transfer_characteristics = nb.u(8)
            matrix_coefficients = nb.u(8)
    chroma_loc_info_present_flag = nb.u(1)
    if chroma_loc_info_present_flag:
        chroma_sample_loc_type_top_field = nb.ue()
        chroma_sample_loc_type_bottom_field = nb.ue()
    timing_info_present_flag = nb.u(1)
    if timing_info_present_flag:
        num_units_in_tick = nb.u(32)
        time_scale = nb.u(32)
        fixed_frame_rate_flag = nb.u(1)
    nal_hrd_parameters_present_flag = nb.u(1)
    if nal_hrd_parameters_present_flag:
        hrd_parameters()
    vcl_hrd_parameters_present_flag = nb.u(1)
    if vcl_hrd_parameters_present_flag:
        hrd_parameters()
    if nal_hrd_parameters_present_flag or vcl_hrd_parameters_present_flag:
        low_delay_hrd_flag = nb.u(1)
    pic_struct_present_flag = nb.u(1)
    bitstream_restriction_flag = nb.u(1)
    if bitstream_restriction_flag:
        motion_vectors_over_pic_boundaries_flag = nb.u(1)
        max_bytes_per_pic_denom = nb.ue()
        max_bits_per_mb_denom = nb.ue()
        log2_max_mv_length_horizontal = nb.ue()
        log2_max_mv_length_vertical = nb.ue()
        max_num_reorder_frames = nb.ue()
        max_dec_frame_buffering = nb.ue()


print(sps)

0x0000016742c00bd902c4e8400000004000000c83c50a9200
{'forbidden_zero_bit': 0, 'nal_ref_idc': 3, 'nal_unit_type': 7}
{'profile_idc': 66, 'level_idc': 11, 'seq_parameter_set_id': 0, 'constraint_set0_flag': 1, 'constraint_set1_flag': 1, 'constraint_set2_flag': 0, 'constraint_set3_flag': 0, 'constraint_set4_flag': 0, 'constraint_set5_flag': 0, 'reserved_zero_2bits': 0}


In [25]:
print(profile_idc)
print(level_idc)

66
11


In [24]:
# IDR
nalu_ba = nalus_ba[4]
print(nalu_ba)
nalu_ba.replace('0x000003', '0x0000', bytealigned=True)
nalu_bs = BitStream(nalu_ba)
nb = H264Bits(nalu_bs)
params = {"forbidden_zero_bit" : nb.f(1),
            "nal_ref_idc" : nb.u(2),
            "nal_unit_type" : nb.u(5)}
print(params)

0x000001419a3812e69fae0b87f260be6c6a19d57f96ba5e5fc8bfdf74d72a2df26259d55f97ac4dd562e5957d7aa0bc718b3625d608440b21a979ecf7d606038bb4d7a97d1e0b4a6c432eb088993f24a35efa72c2229a13813dc2da9fcb9078fcbe2092c40bf09a1477777b45cf04e50a1527146ee734cf9777bdb6858e...
{'forbidden_zero_bit': 0, 'nal_ref_idc': 2, 'nal_unit_type': 1}
